In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import warnings
warnings.filterwarnings('ignore')
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
from datetime import datetime, timedelta
import os
chromedriver = "/Users/stevenwang/Projects/funded-and-hiring-site/newsletter_scraper/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
import pyperclip
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pytest_timeout
import dateparser

In [ ]:
def get_content(url):
    page = requests.get(url)
    return page.content

In [ ]:
ttnews_content = get_content('https://www.ttnews.com/articles')

In [ ]:
fw_news_content = get_content('https://www.freightwaves.com/news/category/trucking')

In [94]:
overdrive_business_content = get_content('https://www.overdriveonline.com/business')

In [ ]:
def get_ttnews(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='views-row')
    
    for article in articles:
        title = article.find('a')
        date = article.find(class_='date-display-single')
        text =  article.find('p') # description
        image = article.find('img')

        if (date != None and title != None and text != None):
            parsed_date = dateparser.parse(date['content'])
            companies.append('TTNews')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append(title.get('href'))
            titles.append(title.getText().strip())
            images.append(image)
            company_urls.append('https://ttnews.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

ttnews = get_ttnews(ttnews_content)

ttnews

In [101]:
def get_overdrive_business(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='node-list__node')
    # print(articles)
    for article in articles:
        title = article.find('h5', class_='section-feed-content-node__content-short-name').a
        # print(title)
        date = article.find(class_='section-feed-content-node__content-published')
        # print(date)
        text =  article.find(class_='section-feed-content-node__content-teaser').a
        # print(text)
        image = article.find('img')

        if (date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Overdrive')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append(title.get('href'))
            titles.append(title.getText().strip())
            images.append(image['src'])
            company_urls.append('https://www.overdriveonline.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

overdrive_business = get_overdrive_business(overdrive_business_content)

overdrive_business

,company,headline,description,publish_date,story_url,company_url,image
0,Overdrive,An edge on growth: One way small fleets stand ...,Could Biden's vax mandate enhance the competit...,2021-09-30,/channel-19/article/15115093/one-way-small-fle...,https://www.overdriveonline.com/,https://img.overdriveonline.com/files/base/ran...
1,Overdrive,Landstar owner-op poised for growth after winn...,Texas-based hauler Mike Lamb said winning the ...,2021-09-30,/life/article/15115187/landstar-ownerop-wins-n...,https://www.overdriveonline.com/,https://img.overdriveonline.com/files/base/ran...
2,Overdrive,Bryan Smith bags the big prize from TCA/Overdrive,The Owner-Operator of the Year was named Tuesd...,2021-09-29,/owner-operator-of-the-year/article/15115116/b...,https://www.overdriveonline.com/,https://img.overdriveonline.com/files/base/ran...
3,Overdrive,Partners in Business tip: Work to improve your...,Do not wait around and accumulate more debt ov...,2020-12-02,/partners-in-business/article/14886575/work-to...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
4,Overdrive,Fuel prices surge to nearly 7-year high,The modest price increase over the past week b...,2021-09-28,/business/article/15115092/fuel-prices-surge-t...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
5,Overdrive,Are owner-operators a legal landmine for fleet...,Recent court rulings cast doubt on the future ...,2021-09-28,/home/article/15115082/are-owneroperators-a-le...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
6,Overdrive,FMCSA shuts down New York-based trucker after ...,Bobir M. Kholmurodov has been effectively shut...,2021-09-27,/regulations/article/15115058/new-yorkbased-tr...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
7,Overdrive,Breaking Free: The history of the owner-operat...,From fighting for freedom to haul in its early...,2011-09-01,/business/article/14876086/breaking-free-a-bri...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
8,Overdrive,Tractor-trailers banned from section of I-95; ...,DelDot urges truckers to use I-495 as an alter...,2021-09-23,/home/article/15114977/tractortrailers-banned-...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."
9,Overdrive,DOT asks for truckers' input on supply chain d...,DOT will use information from the trucking ind...,2021-09-22,/regulations/article/15114954/dot-asks-trucker...,https://www.overdriveonline.com/,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK..."


In [ ]:
def get_freightWaves(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []

    articles = soup.find_all(class_='post-item')
    
    for article in articles:
        title = article.find('a')
        date =  article.find(class_='date meta-item tie-icon')
        text =  article.find('p')
        image = article.find('img')

        companies.append('FreightWaves')
        description.append(text.getText())
        publish_date.append(dateparser.parse(date.get_text()).date())
        stories.append(title.get('href'))
        titles.append(title.get('aria-label'))
        images.append(image)
        company_urls.append('https://www.freightwaves.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe
        
fw_news = get_freightWaves(fw_news_content)

fw_news

In [ ]:
final = pd.concat([ttnews, fw_news, overdrive_business], axis = 0)
#replace all .coms in company names
final['company'] = final.company.str.replace("[.].*|•|’|,|[+]|[()]|[#]", '').str.strip()

In [ ]:
final

In [ ]:
final.head()

In [ ]:
len(final)

In [ ]:
#check previous posts, check to see there are no duplicates
# old = pd.read_csv('historical_deals.csv')
# dupes = pd.merge(final, old[['company', 'deal_size']], how = 'inner', on = ['company', 'deal_size'])
# final = final[~final.company.isin(dupes.company)]

In [ ]:
len(final)

In [ ]:
final

In [ ]:
#save deal records
# new_old = pd.concat([final, old], axis = 0)
new_old = pd.concat([final], axis = 0)

new_old.to_csv('historical_deals.csv', index = False)

In [ ]:
len(new_old)

In [ ]:
date = 'Sunday, 08/29/2021'

title = '''
<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>{date}</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>
'''.format(date = date).strip()

In [ ]:
print(title)
pyperclip.copy(title)

In [ ]:
content = ''''''

final_dict = final.to_dict(orient = 'records')
final_dict = final_dict[0:7]
for row in final_dict:
    headline = row['headline']
    company_name = row['company']
    publish_date = row['publish_date']
    description = row['description']
    story_url = row['story_url']
    company_url = row['company_url']
    img = row['image']

    row_url = company_url + story_url
    
    entry = '''
    <tr>
      <td>
        {img}
        <h3>
          <a href = "{row_url}">{headline}</a>
        </h3>
        <p>
          {description}
        </p>
        <p>
          <a href = "{company_url}">{company_name}</a>
        </p>
        <p>
          <b>{publish_date}</b>
        </p>
      </td>
    </tr> '''.format(row_url = row_url, headline = headline, company_url = company_url, company_name = company_name, description = description, publish_date = publish_date, img=img).strip()

    content += '\n' + entry.strip()

In [ ]:
pyperclip.copy(content)
print(content)

In [ ]:
#job boards
# stackoverflow, jobvite, bamboohr